In [2]:
## Đọc dữ liệu từ tệp

import math
import numpy as np
with open('./data/vidu3_lin_reg.txt') as f:
    lines = f.readlines()
x_data = []
y_data = []

lines.pop(0)

for line in lines:
    splitted = line.split()
    # ID.append(float(splitted[0]))
    # TUOI.append(float(splitted[1]))
    # BMI.append(float(splitted[2]))
    # HA.append(float(splitted[3]))
    # GLUCOSE.append(float(splitted[4]))
    # CHOLESTEROL.append(float(splitted[5]))
    # BEDAYNTM.append(float(splitted[6]))
    x_data.append([splitted[1], splitted[5], splitted[4], splitted[3], splitted[2]])
    y_data.append(splitted[6])

y_data = np.asarray(y_data, dtype=float)
x_data = np.asarray(x_data, dtype=float)


In [3]:
## Trường hợp sử dụng Numpy – Dùng thuật toán HoldHouse để khai triển QR

def qr_householder(A):
    #""" Compute QR decomposition of A using Householder reflection"""
    M = A.shape[0]
    N = A.shape[1]

    # set Q to the identity matrix
    Q = np.identity(M)

    # set R to zero matrix
    R = np.copy(A)

    for n in range(N):
        # vector to transform
        x = A[n:, n]
        k = x.shape[0]

        # compute ro=-sign(x0)||x||
        ro = -np.sign(x[0]) * np.linalg.norm(x)

        # compute the householder vector v
        e = np.zeros(k)
        e[0] = 1
        v = (1 / (x[0] - ro)) * (x - (ro * e))

        # apply v to each column of A to find R
        for i in range(N):
            R[n:, i] = R[n:, i] - (2 / (v@v)) * ((np.outer(v, v)) @ R[n:, i])

        # apply v to each column of Q
        for i in range(M):
            Q[n:, i] = Q[n:, i] - (2 / (v@v)) * ((np.outer(v, v)) @ Q[n:, i])

    return Q.transpose(), R

def linear_regression(x_data, y_data):
# """
# This function calculate linear regression base on x_data and y_data
# :param x_data: vector
# :param y_data: vector
# :return: w (regression estimate)
# """

# add column 1
    x_bars = np.concatenate((np.ones((x_data.shape[0], 1)), x_data), axis=1)

    Q, R = qr_householder(x_bars) # QR decomposition
    R_pinv = np.linalg.pinv(R) # calculate inverse matrix of R
    A = np.dot(R_pinv, Q.T) # apply formula

    return np.dot(A, y_data)

In [26]:
# Chia dữ liệu thành 80 dòng đầu cho training và 20 dòng sau cho testing
x_train = x_data[:80]
y_train = y_data[:80]
x_test = x_data[80:]
y_test = y_data[80:]
w = linear_regression(x_data, y_data) # get result
w = w.T.tolist()
print("Hệ số hồi quy:(Toàn bộ dữ liệu)")
line = ['Intercept', 'TUOI', "CHOLESTEROL", "GLUCOSE", 'HA', 'BMI']
res = list(zip(line, w))
for o in res:
    print("{: >20}: {: >10}".format(*o))
    
w = linear_regression(x_train, y_train) # get result
w = w.T.tolist()
print("----------------------------------------")
print("Hệ số hồi quy:(80 train - 20 test)")
line = ['Intercept', 'TUOI', "CHOLESTEROL", "GLUCOSE", 'HA', 'BMI']
res = list(zip(line, w))
for o in res:
    print("{: >20}: {: >10}".format(*o))
    
# Thêm cột 1 vào x_data
x_test_with_intercept = np.concatenate((np.ones((x_test.shape[0], 1)), x_test), axis=1)

# Tính dự đoán y_pred
y_pred = np.dot(x_test_with_intercept, w)

# Tính các đại lượng kỳ vọng và phương sai của sai số
residuals = y_test - y_pred
mean_residuals = np.mean(residuals)
var_residuals = np.var(residuals)

# In kết quả
print("Đại lượng kỳ vọng của sai số:", mean_residuals)
print("Phương sai của sai số:", var_residuals)


Hệ số hồi quy:(Toàn bộ dữ liệu)
           Intercept: -0.10802908119386984
                TUOI: 0.009946940947109876
         CHOLESTEROL: 0.007670101694229964
             GLUCOSE: 0.002181366945247507
                  HA: 0.03847024823970345
                 BMI: -0.00020654770133402
----------------------------------------
Hệ số hồi quy:(80 train - 20 test)
           Intercept: 0.043064364103294756
                TUOI: 0.008989196889296783
         CHOLESTEROL: 0.008086342231978162
             GLUCOSE: 0.0026021798675557803
                  HA: 0.0070853523419238205
                 BMI: -0.00047742422185274164
Đại lượng kỳ vọng của sai số: 0.07998861696197208
Phương sai của sai số: 0.22100281283267012


In [22]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Đọc dữ liệu từ tệp văn bản
with open('./data/vidu3_lin_reg.txt') as f:
    lines = f.readlines()

# Loại bỏ tiêu đề
lines.pop(0)

# Khởi tạo danh sách các biến và nhãn
x_data = []
y_data = []

for line in lines:
    splitted = line.split()
    x_data.append([float(splitted[1]), float(splitted[4]), float(splitted[3]), float(splitted[5]), float(splitted[2])])
    y_data.append(float(splitted[6]))

# Chuyển danh sách thành mảng numpy
x_data = np.array(x_data)
y_data = np.array(y_data)

# Xây dựng mô hình hồi quy tuyến tính với toàn bộ dữ liệu
model = LinearRegression().fit(x_data, y_data)

# In ra các hệ số hồi quy
print("Hệ số hồi quy:(Toàn bộ dữ liệu)")
print("Beta0 (Intercept):", model.intercept_)
print("Beta1 (Tuổi):", model.coef_[0])
print("Beta2 (Cholesterol):", model.coef_[1])
print("Beta3 (Glucose):", model.coef_[2])
print("Beta4 (Huyết áp TT):", model.coef_[3])
print("Beta5 (BMI):", model.coef_[4])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra (80% huấn luyện, 20% kiểm tra)
# x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
# Chia dữ liệu thành 80 dòng đầu cho training và 20 dòng sau cho testing
x_train = x_data[:80]
y_train = y_data[:80]
x_test = x_data[80:]
y_test = y_data[80:]

# Huấn luyện mô hình trên tập huấn luyện
model.fit(x_train, y_train)

# In ra các hệ số hồi quy
print("----------------------------------------")
print("Hệ số hồi quy:(80 train - 20 test)")
print("Beta0 (Intercept):", model.intercept_)
print("Beta1 (Tuổi):", model.coef_[0])
print("Beta2 (Cholesterol):", model.coef_[1])
print("Beta3 (Glucose):", model.coef_[2])
print("Beta4 (Huyết áp TT):", model.coef_[3])
print("Beta5 (BMI):", model.coef_[4])

# Dự đoán trên tập kiểm tra
y_pred = model.predict(x_test)

# Tính các đại lượng kỳ vọng và phương sai của sai số trên tập kiểm tra
residuals = y_test - y_pred
mean_residuals = np.mean(residuals)
var_residuals = np.var(residuals)

# In kết quả
print("Đại lượng kỳ vọng của sai số trên tập kiểm tra:", mean_residuals)
print("Phương sai của sai số trên tập kiểm tra:", var_residuals)

Hệ số hồi quy:(Toàn bộ dữ liệu)
Beta0 (Intercept): -0.10802908119386845
Beta1 (Tuổi): 0.009946940947109854
Beta2 (Cholesterol): 0.007670101694230011
Beta3 (Glucose): 0.0021813669452475024
Beta4 (Huyết áp TT): 0.03847024823970346
Beta5 (BMI): -0.00020654770133401464
----------------------------------------
Hệ số hồi quy:(80 train - 20 test)
Beta0 (Intercept): 0.0430643641032975
Beta1 (Tuổi): 0.008989196889296787
Beta2 (Cholesterol): 0.008086342231978147
Beta3 (Glucose): 0.0026021798675557707
Beta4 (Huyết áp TT): 0.007085352341923779
Beta5 (BMI): -0.00047742422185274267
Đại lượng kỳ vọng của sai số trên tập kiểm tra: 0.07998861696197088
Phương sai của sai số trên tập kiểm tra: 0.22100281283267015
